In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from keras.models import load_model

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
actions = np.array(['Indian', 'Sign', 'Language', 'Man', 'Woman', 'Namaste', 'Deaf', 'Sorry', 'Happy', 'Sad', 'Understand', 'Bye', 'Please', 'Food', 'Water'])

In [ ]:
model = load_model('actions_95%_0.1_1000.h5')

In [ ]:
# accuracy=[]
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.9, min_tracking_confidence=0.9) as holistic:
    while cap.isOpened():

     
        ret, frame = cap.read()

       
        image, results = mediapipe_detection(frame, holistic)
        
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
        
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold:
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            # accuracy.append(str(round(int(res[np.argmax(res)]*100),2)))
                    else:
                        sentence.append(actions[np.argmax(res)])
                        # accuracy.append(str(round(int(res[np.argmax(res)]*100),2)))

        if len(sentence) > 1: 
            sentence = sentence[-1:]
            # accuracy = accuracy[-1:]
        
        string = str(sentence);  
        count = 0;  
        
        for i in range(0, len(string)):  
            if(string[i] != ' '):  
                count = count + 1
        
        if (count==7):
            count = count + 1
        elif (count==14):
            count = count - 2   

            
        cv2.rectangle(image, (0,0), (count*15, 40), (245, 117, 16), -1)
        # cv2.rectangle(image, (0,0), (count*30, 40), (245, 117, 16), -1)
        cv2.putText(image,''.join(sentence), (15,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # cv2.putText(image,''.join(sentence)+' - '+''.join(accuracy)+'%', (15,30), 
        #                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
       
        cv2.imshow('OpenCV Feed', image)

        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()